In [35]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install ee wxee rasterio

In [37]:
import os
import sys

import pandas as pd
import numpy as np

import xarray as xr
import geopandas as gpd

import pickle
import pyproj
import ee
import wxee
import rasterio

In [38]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file='/content/drive/MyDrive/w/private-key.json'
)

ee.Initialize(credentials)

In [39]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

In [40]:
def mask_nc_file(
    nc,
    variable,
    lat_min,
    lat_max,
    lon_min,
    lon_max
):
    with xr.open_dataset(nc) as xr_nc:
        result = xr_nc.sel(
            lat = slice(lat_max, lat_min),
            lon = slice(lon_min, lon_max),
        )
    return result

In [41]:
datasets_P = mask_nc_file(
    nc = "/content/drive/MyDrive/w/Past/P/Daily/1979001.nc",
    variable = "precipitation",
    lat_min = LAT_MIN,
    lat_max = LAT_MAX,
    lon_min = LON_MIN,
    lon_max = LON_MAX
)
datasets_P

<xarray.Dataset>
Dimensions:        (lon: 600, lat: 200, time: 1)
Coordinates:
  * lon            (lon) float32 -124.9 -124.8 -124.8 ... -65.25 -65.15 -65.05
  * lat            (lat) float32 44.95 44.85 44.75 44.65 ... 25.25 25.15 25.05
  * time           (time) datetime64[ns] 1979-01-01
Data variables:
    precipitation  (time, lat, lon) float32 ...
Attributes:
    history:  Created on 2021-01-12 22:09

In [43]:
crs = 4326
shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])




soil_era_evaporation = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_1')

soil_era_transpiration1 = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_2')

soil_era_transpiration2 = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_3')

soil_era_transition = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR").filter(ee.Filter.date('1978-12-31', '1979-01-01')).select('volumetric_soil_water_layer_4')


soil_era_evaporation = soil_era_evaporation.map(lambda image: image.clip(shape))
soil_era_transpiration1 = soil_era_transpiration1.map(lambda image: image.clip(shape))
soil_era_transpiration2 = soil_era_transpiration2.map(lambda image: image.clip(shape))
soil_era_transition = soil_era_transition.map(lambda image: image.clip(shape))

soil_era_evaporation = soil_era_evaporation.set("system:time_start", ee.Date("1979-01-01"))
soil_era_transpiration1 = soil_era_transpiration1.set("system:time_start", ee.Date("1979-01-01"))
soil_era_transpiration2 = soil_era_transpiration2.set("system:time_start", ee.Date("1979-01-01"))
soil_era_transition = soil_era_transition.set("system:time_start", ee.Date("1979-01-01"))


arr_soil_era_evaporation = soil_era_evaporation.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')
arr_soil_era_transpiration1 = soil_era_transpiration1.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')
arr_soil_era_transpiration2 = soil_era_transpiration2.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')
arr_soil_era_transition = soil_era_transition.wx.to_xarray(scale=11132, crs=f'EPSG:{crs}')

da_soil_era_evaporation = arr_soil_era_evaporation.rio.write_crs(f"epsg:{crs}", inplace=True)
da_soil_era_transpiration1 = arr_soil_era_transpiration1.rio.write_crs(f"epsg:{crs}", inplace=True)
da_soil_era_transpiration2 = arr_soil_era_transpiration2.rio.write_crs(f"epsg:{crs}", inplace=True)
da_soil_era_transition = arr_soil_era_transition.rio.write_crs(f"epsg:{crs}", inplace=True)



datasets_P = datasets_P.rename({'lon': 'x','lat': 'y'})
datasets_P1 = datasets_P.rio.write_crs(f"epsg:{crs}", inplace=True)


xds_repr_match__soil_era_evaporation = da_soil_era_evaporation.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)
xds_repr_match__soil_era_transpiration1 = da_soil_era_transpiration1.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)
xds_repr_match__soil_era_transpiration2 = da_soil_era_transpiration2.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)
xds_repr_match__soil_era_transition = da_soil_era_transition.rio.reproject_match(datasets_P1, resampling = rasterio.enums.Resampling.nearest)


xds_repr_match__soil_era_evaporation = xds_repr_match__soil_era_evaporation * (70-0)
xds_repr_match__soil_era_transpiration1 = xds_repr_match__soil_era_transpiration1 * (280 - 70)
xds_repr_match__soil_era_transpiration2 = xds_repr_match__soil_era_transpiration2 * (1000 - 280)
xds_repr_match__soil_era_transition = xds_repr_match__soil_era_transition * (2890-1000)

xds_repr_match__soil_era_evaporation

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/1 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:                        (x: 600, y: 200, time: 1)
Coordinates:
  * x                              (x) float32 -124.9 -124.8 ... -65.15 -65.05
  * y                              (y) float32 44.95 44.85 44.75 ... 25.15 25.05
  * time                           (time) datetime64[ns] 1978-12-31
    spatial_ref                    int64 0
Data variables:
    volumetric_soil_water_layer_1  (time, y, x) float64 nan nan nan ... nan nan
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    _FillValue:              -32768.0
    scale_factor:            1.0
    add_offset:              0.0

In [44]:
# Convert time coordinate to NumPy datetime64 type
time_values_evap = np.asarray(xds_repr_match__soil_era_evaporation['time'].values, dtype='datetime64[D]')
time_values_transp1 = np.asarray(xds_repr_match__soil_era_transpiration1['time'].values, dtype='datetime64[D]')
time_values_transp2 = np.asarray(xds_repr_match__soil_era_transpiration2['time'].values, dtype='datetime64[D]')
time_values_trans = np.asarray(xds_repr_match__soil_era_transition['time'].values, dtype='datetime64[D]')

# Subtract one day from each time value
updated_time_values_evap = time_values_evap + np.timedelta64(1, 'D')
updated_time_values_transp1 = time_values_transp1 + np.timedelta64(1, 'D')
updated_time_values_transp2 = time_values_transp2 + np.timedelta64(1, 'D')
updated_time_values_trans = time_values_trans + np.timedelta64(1, 'D')

# Update the time coordinate with the modified values
xds_repr_match__soil_era_evaporation['time'] = updated_time_values_evap
xds_repr_match__soil_era_transpiration1['time'] = updated_time_values_transp1
xds_repr_match__soil_era_transpiration2['time'] = updated_time_values_transp2
xds_repr_match__soil_era_transition['time'] = updated_time_values_trans

In [45]:
merged_dataset = xr.merge([xds_repr_match__soil_era_evaporation, xds_repr_match__soil_era_transpiration1, xds_repr_match__soil_era_transpiration2, xds_repr_match__soil_era_transition])

merged_dataset.to_netcdf("/content/drive/MyDrive/w/soil_water_content_1979001.nc")